In [0]:
!pip3 install catalyst
!pip3 install pretrainedmodels
!pip3 install git+https://github.com/qubvel/segmentation_models.pytorch
!pip3 install pytorch_toolbelt
!pip3 install torchvision==0.4

    100% |████████████████████████████████| 286kB 792kB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 403kB/s ta 0:00:01
    100% |████████████████████████████████| 460kB 465kB/s ta 0:00:01
    100% |████████████████████████████████| 28.7MB 36kB/s eta 0:00:01   24% |███████▊                        | 7.0MB 376kB/s eta 0:00:58    45% |██████████████▍                 | 12.9MB 517kB/s eta 0:00:31    86% |███████████████████████████▊    | 24.8MB 594kB/s eta 0:00:07
    100% |████████████████████████████████| 3.3MB 138kB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 46kB/s ta 0:00:0101
    100% |████████████████████████████████| 26.5MB 38kB/s eta 0:00:01    42% |█████████████▋                  | 11.3MB 428kB/s eta 0:00:36    77% |████████████████████████▉       | 20.6MB 227kB/s eta 0:00:26
    100% |████████████████████████████████| 7.2MB 41kB/s ta 0:00:011
    100% |████████████████████████████████| 3.8MB 200kB/s ta 0:00:01
    100% |████████████████████████

    100% |████████████████████████████████| 81kB 217kB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 269kB/s ta 0:00:01
    100% |████████████████████████████████| 327kB 64kB/s ta 0:00:011
    100% |████████████████████████████████| 2.3MB 236kB/s ta 0:00:01
    100% |████████████████████████████████| 112kB 404kB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 38kB/s ta 0:00:01
    100% |████████████████████████████████| 153kB 58kB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 171kB/s ta 0:00:01
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/ochir/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /home/ochir/.cache/pip/wheels/a7/15/a0/0a0561549ad11cdc1bc8fa1191a353efd30facf6bfb507aefc
Successfully built retrying absl-py
tensorboard 2.0.1 has requirement setuptools>=41.0.0, but you'll have setuptools 40.8.

    100% |████████████████████████████████| 8.8MB 92kB/s eta 0:00:01    45% |██████████████▍                 | 4.0MB 203kB/s eta 0:00:24
  Running setup.py bdist_wheel for segmentation-models-pytorch ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-wvk5k19a/wheels/79/3f/09/1587a252e0314d26ad242d6d2e165622ab95c95e5cfe4b942c
  Running setup.py bdist_wheel for efficientnet-pytorch ... done
  Stored in directory: /home/ochir/.cache/pip/wheels/27/56/13/5bdaa98ca8bd7d5da65cc741987dd14391b87fa1a09081d17a
Successfully built segmentation-models-pytorch efficientnet-pytorch
torchvision 0.4.0 has requirement torch==1.2.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torchvision 0.4.1
    Uninstalling torchvision-0.4.1:
      Successfully uninstalled torchvision-0.4.1
    100% |████████████████████████████████| 71kB 488kB/s ta 0:00:011
  Running setup.py bdist_wheel for pytorch-toolbelt ... done
  Stored in directory: /home/ochir/.cache/pip/wheels/51/

In [0]:
import os
import cv2
import collections
import time
import tqdm
from PIL import Image
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
from albumentations import pytorch as AT

import segmentation_models_pytorch as smp

from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst.dl.runner import SupervisedRunner
from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback

In [0]:
def get_img(x, folder: str='train'):
    #return image based on image name and folder.
    data_folder = f'{path}/{folder}'
    image_path = os.path.join(data_folder,x)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [0]:
def rle_decode(mask_rle: str = '', shape: tuple = (1400, 2100)):
    #Decode rle encode mask.
    #param mask_rle: run-length as string formatted (start length)
    #param shape: (height, width) of array to return
    #Returns numpy array, 1 - mask, 0 - background
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')    

In [0]:
def make_mask(df: pd.DataFrame, image_name: str='img.jpg', shape: tuple = (1400, 2100)):
    #Create mask based on df, image name and shape
    encoded_masks = df.loc[df['im_id'] == image_name, 'EncodedPixels']
    masks = np.zeros((shape[0], shape[1], 4), dtype=np.float32)
    
    for idx, label in enumerate(encoded_masks.values):
        if(label is not np.nan):
            mask = rle_decode(label)
            masks[:, :, idx] = mask
    return masks

In [0]:
def to_tensor(x, **kwargs):
    #Convert image or mask
    return x.transpose(2,0,1).astype('float32')

In [0]:
def mask2rle(img):
#     convert mask to rle
#     img: numpy array, 1 - mask, 0 - background
#     returns run length as string formatted
    pixels=img.T.flatten()
    pixels=np.concatenate([[0], pixels,[0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0]+1
    runs[1:2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [0]:
def visualize(image, mask, original_image=None, original_mask=None):
    #Plot image and masks
    #If two pairs of images and masks are passes, show both
    fontsize = 14
    class_dict = {0: 'Fish', 1: 'Flower', 2: 'Gravel', 3: 'Sugar'}
    
    if(original_image is None and original_mask is None):
        f, ax =plt.subplot(1,5,figsize=(24,24))
        
        ax[0,0].imshow(original_image)
        ax[0,0].set_title('Original image', fontsize=fontsize)
        
        for i in range(4):
            ax[0, i + 1].imshow(original_mask[:,:,i])
            ax[0, i + 1].set_title(f'Original mask {class_dict[i]}', fontsize=fontsize)
            
        ax[1, 0].imshow(image)
        ax[1, 0].set_title('Transformed image', fontsize=fontsize)
        
        for i in range(4):
            ax[1,i + 1].imshow(mask[:,:,i])
            ax[1,i + 1].set_title(f'Transformed mask {class_dict[i]}', fontsize=fontsize)

In [0]:
def visualize_with_raw(image, mask, original_image=None, original_mask=None, raw_image=None,raw_mask=None):
#     Plot image and masks
#     if two pairs of images and masks are passes show both
    fontsize = 14
    class_dict = {0: 'Fish', 1: 'Flower', 2: 'Gravel', 3: 'Sugar'}
    
    f, ax = plt, subplots(3,5, figsize=(24,12))
    
    ax[0, 0].imshow(original_image)
    ax[0, 0].set_title('Original image', fontsize=fontsize)

    for i in range(4):
        ax[0, i + 1].imshow(original_mask[:, :, i])
        ax[0, i + 1].set_title(f'Original mask {class_dict[i]}', fontsize=fontsize)


    ax[1, 0].imshow(raw_image)
    ax[1, 0].set_title('Original image', fontsize=fontsize)

    for i in range(4):
        ax[1, i + 1].imshow(raw_mask[:, :, i])
        ax[1, i + 1].set_title(f'Raw predicted mask {class_dict[i]}', fontsize=fontsize)
        
    ax[2, 0].imshow(image)
    ax[2, 0].set_title('Transformed image', fontsize=fontsize)


    for i in range(4):
        ax[2, i + 1].imshow(mask[:, :, i])
        ax[2, i + 1].set_title(f'Predicted mask with processing {class_dict[i]}', fontsize=fontsize)

In [0]:
def plot_with_augmentation(image, mask, augment):
    